In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
import pickle
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
# pre
data_dir = "../../data/"
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
training_data_list = load_obj("SBdata_all_feature-v2")
test_data = load_obj("SB_testData_all_feature-V2")

# Get the training_data_df
training_data_df = pd.DataFrame(training_data_list) 
final_training_data_df = training_data_df.iloc[:,2:20]
# 0.511756 0.028259
# get the labeled data df
final_labels_df = training_data_df.iloc[:,1]
test_data_df = pd.DataFrame(test_data) 

In [4]:
# final_training_data_df = rescale_min_max(final_training_data_df)
X=final_training_data_df
# count=0
# get the data and label
y=final_labels_df

from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X,y)
x_train = X_t
x_test = X_test
y_train = y_t
y_test = y_test

In [8]:
def create_model():
    activation='relu'
    dropout_rate=0.0
#     init_mode='uniform'
#     weight_constraint=0
    optimizer='SGD'
    model = Sequential()
    model.add(Dense(128, input_dim=18, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation=activation))
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model


In [ ]:
model = KerasClassifier(build_fn=create_model, batch_size=200, epochs=200)
activation=['relu','tanh','sigmoid','hard_sigmoid','linear']
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
epochs = [50, 100, 200]
batch_size = [100, 250, 500]
param_grid = dict(epochs=epochs, batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(x_train, y_train)


# summarise results
print('Best: %f using %s'%(grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, params in zip(means, stds, params):
    print("%f (%f) with: %r"%(mean, stdev, param))

Epoch 1/50
Epoch 1/50
150000/150000 [==============================] - 10s 68us/step - loss: 7.9505 - acc: 0.0016
Epoch 2/50
150000/150000 [==============================] - 11s 72us/step - loss: 7.8896 - acc: 0.0135
Epoch 2/50
150000/150000 [==============================] - 9s 63us/step - loss: 7.9694 - acc: 2.8000e-04
Epoch 3/50
150000/150000 [==============================] - 10s 66us/step - loss: 7.9414 - acc: 0.0000e+00
Epoch 3/50
150000/150000 [==============================] - 9s 63us/step - loss: 7.9694 - acc: 2.8000e-04
Epoch 4/50
150000/150000 [==============================] - 9s 61us/step - loss: 7.9414 - acc: 0.0000e+00
Epoch 4/50
150000/150000 [==============================] - 10s 67us/step - loss: 7.9694 - acc: 2.8000e-04
Epoch 5/50
150000/150000 [==============================] - 9s 59us/step - loss: 7.9694 - acc: 2.8000e-04
Epoch 6/50
150000/150000 [==============================] - 9s 59us/step - loss: 7.9694 - acc: 2.8000e-04
Epoch 7/50
150000/150000 [=============

150000/150000 [==============================] - 10s 64us/step - loss: 7.9414 - acc: 0.0000e+00
Epoch 34/50
150000/150000 [==============================] - 10s 65us/step - loss: 7.9694 - acc: 2.8000e-04
Epoch 35/50
150000/150000 [==============================] - 10s 64us/step - loss: 7.9414 - acc: 0.0000e+00
Epoch 35/50
150000/150000 [==============================] - 10s 65us/step - loss: 7.9414 - acc: 0.0000e+00
Epoch 36/50
150000/150000 [==============================] - 9s 63us/step - loss: 7.9694 - acc: 2.8000e-04
Epoch 38/50
150000/150000 [==============================] - 9s 62us/step - loss: 7.9414 - acc: 0.0000e+00
Epoch 38/50
150000/150000 [==============================] - 9s 62us/step - loss: 7.9694 - acc: 2.8000e-04
Epoch 39/50
150000/150000 [==============================] - 9s 59us/step - loss: 7.9414 - acc: 0.0000e+00
Epoch 39/50
150000/150000 [==============================] - 9s 63us/step - loss: 7.9414 - acc: 0.0000e+00
Epoch 40/50
150000/150000 [==================

In [12]:
grid_search = grid
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
predictions = grid_search.predict(X_test)
print('Accuracy:', accuracy_score(y_test, predictions))
print('Precision:', precision_score(y_test, predictions))
print('Recall:', recall_score(y_test, predictions))

Best score: 0.499
Best parameters set:


NameError: name 'parameters' is not defined

In [10]:

def predict():
    """
    make the prediction using the jaccard's coefficient
    """
    result = []
    cnt=0
    for row in test_data_df.iterrows():
        row_df = pd.DataFrame(row[1]).T
        single_result = grid_result.predict(row_df)[0][0]
#         print(model.predict(row_df)[0][0])
        cnt+=1
        result.append((cnt, single_result))
    return result
result = predict()

In [13]:
result

[(1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 1),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 1),
 (99, 1),
 (100, 1),
 (101, 1